In [2]:
import requests
import json
import urllib.request
import pandas as pd
import ast

In [22]:
response_details_merge = pd.read_csv('../data/Data 6 - Merged Responses and Movie Details.csv')
response_details_merge.head()

,Unnamed: 0,id_x,person_mbti,mbti_test,character,movie,new_title,genre_ids,id_y,popularity,release_date,title,vote_average,vote_count
0,0,0,ENFP,michaelcaloz.com,Tony Stark,Avengers: Endgame,Avengers: Endgame,"[12, 878, 28]",299534,240.886,2019-04-24,Avengers: Endgame,8.255,24867
1,1,0,ENFP,michaelcaloz.com,Rocket Raccoon,Guardians of the Galaxy Vol. 3,Guardians of the Galaxy Vol. 3,"[878, 12, 28]",447365,169.896,2023-05-03,Guardians of the Galaxy Vol. 3,7.972,6262
2,2,0,ENFP,michaelcaloz.com,Captain America,Avengers: Endgame,Avengers: Endgame,"[12, 878, 28]",299534,240.886,2019-04-24,Avengers: Endgame,8.255,24867
3,3,0,ENFP,michaelcaloz.com,Peter Parker,Spider-Man: Far From Home,Spider-Man: Far From Home,"[28, 12, 878]",429617,124.720,2019-06-28,Spider-Man: Far From Home,7.400,15144
4,4,0,ENFP,michaelcaloz.com,Hobie Brown,Spider-Man: Across the Spider-Verse,Spider-Man: Across the Spider-Verse,"[16, 28, 12, 878]",569094,187.181,2023-05-31,Spider-Man: Across the Spider-Verse,8.366,6265


# Actions
### Columns to be removed
- `Unnamed: 0`
- `movie` and `title` since `new_title` was assigned to hold the actual movie title in Script 4
- `vote_count`
### Renames
- `id_x` as `person_id`
- `new_title` as `movie`
- `id_y` as `movie_id`
- `popularity` as `movie_popularity`
- `vote_average` as `movie_vote_avg`
### Others
- Change `release_date` to only capture year information as `movie_year`


In [23]:
response_details = response_details_merge.copy()

response_details.drop(columns = ['Unnamed: 0', 'person_mbti', 'movie', 'title', 'vote_count'], inplace=True)
response_details.rename(columns = {'id_x':'person_id', 'new_title':'movie', 'id_y':'movie_id', 'popularity':'movie_popularity', 'vote_average':'movie_vote_avg'}, inplace = True)
response_details['movie_year'] = response_details_merge['release_date'].astype(str).str[:4]
response_details.drop(columns = ['release_date'], inplace=True)

response_details.head()

,person_id,mbti_test,character,movie,genre_ids,movie_id,movie_popularity,movie_vote_avg,movie_year
0,0,michaelcaloz.com,Tony Stark,Avengers: Endgame,"[12, 878, 28]",299534,240.886,8.255,2019
1,0,michaelcaloz.com,Rocket Raccoon,Guardians of the Galaxy Vol. 3,"[878, 12, 28]",447365,169.896,7.972,2023
2,0,michaelcaloz.com,Captain America,Avengers: Endgame,"[12, 878, 28]",299534,240.886,8.255,2019
3,0,michaelcaloz.com,Peter Parker,Spider-Man: Far From Home,"[28, 12, 878]",429617,124.720,7.400,2019
4,0,michaelcaloz.com,Hobie Brown,Spider-Man: Across the Spider-Verse,"[16, 28, 12, 878]",569094,187.181,8.366,2023


In [24]:
char_mbti_df = pd.read_csv('../data/Data 8 - Franchise-expanded Movies from Data 2.csv')
char_mbti_df

,char,movie,mbti,votes
0,Guido Anselmi,8 1/2,INTP,57
1,King Leonidas of Sparta,300,ENTJ,460
2,Artemisia,300,ENTJ,127
3,Xerxes,300,INTJ,103
4,Gorgo,300,INFJ,62
...,...,...,...,...
10718,Gazelle,Zootopia (2016),ESFP,107
10719,Mr Manchas,Zootopia (2016),ISFJ,40
10720,Doug Ramses,Zootopia (2016),INTP,30
10721,Travis,Zootopia (2016),ESTP,27
